In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [25]:
from dataclasses import dataclass
from math import radians

import numpy as np
import matplotlib.pyplot as plt
from cw.context import time_it
from cw.simulation import Simulation, StatesBase, AB3Integrator, LastValueLogger, Logging

from topone.environment import Environment, Stage
from topone.pid_agent import PIDAgent

In [3]:
@dataclass
class States(StatesBase):
    t: float = 0
    command_engine_on: bool = False
    command_drop_stage: bool = False
    gii: np.ndarray = np.zeros(2)
    xii: np.ndarray = np.zeros(2)
    vii: np.ndarray = np.zeros(2)
    aii: np.ndarray = np.zeros(2)
    fii_thrust: np.ndarray = np.zeros(2)
    theta: float = 0
    theta_dot: float = 0
    mass: float = 0
    mass_dot: float = 0
    h: float = 0
    engine_on: bool = False
    stage_state: int = 0
    stage_idx: int = 0

    def get_y_dot(self):
        y = np.empty(6)
        y[:2] = self.vii
        y[2:4] = self.aii
        y[4] = self.theta_dot
        y[5] = self.mass_dot
        return y

    def get_y(self):
        y = np.empty(6)
        y[:2] = self.xii
        y[2:4] = self.vii
        y[4] = self.theta
        y[5] = self.mass
        return y

    def set_t_y(self, t, y):
        self.t = t
        self.xii = y[:2]
        self.vii = y[2:4]
        self.theta = y[4]
        self.mass = y[5]

In [56]:
simulation = Simulation(
    states_class=States,
    integrator=AB3Integrator(
        h=0.1,
        rk4=True,
        fd_max_order=1),
    modules=[
        Environment(
            surface_diameter=1737.4e3,
            mu=4.9048695e12,
            stages=(
                Stage(1, 1, 100, 2*1.7),
            ),
            initial_altitude=1000,
            initial_theta=radians(5),
            initial_latitude=radians(90),
        ),
        PIDAgent(1, 0, 0)
    ],
    logging=Logging(),
    initial_state_values=None,
)

simulation.initialize()

In [57]:
with time_it("simulation run"):
    result = simulation.run(1000)
result

simulation run: 0.19002002799970796 [s]


<xarray.Dataset>
Dimensions:             (d_2_0: 2, t: 1000)
Coordinates:
  * t                   (t) float64 0.0 0.1 0.2 0.3 0.4 ... 99.6 99.7 99.8 99.9
Dimensions without coordinates: d_2_0
Data variables:
    command_engine_on   (t) bool False False False False ... True True True True
    command_drop_stage  (t) bool False False False False ... False False False
    gii                 (t, d_2_0) float64 -9.938e-17 -1.623 ... -1.622
    xii                 (t, d_2_0) float64 1.064e-10 1.738e+06 ... 1.739e+06
    vii                 (t, d_2_0) float64 0.0 0.0 -9.938e-18 ... 0.8894 21.95
    aii                 (t, d_2_0) float64 -9.938e-17 -1.623 ... 0.4294
    fii_thrust          (t, d_2_0) float64 0.0 0.0 0.0 0.0 ... 3.4 0.0001683 3.4
    theta               (t) float64 0.08727 -0.06109 0.1021 ... 1.571 1.571
    theta_dot           (t) float64 -1.484 1.632 1.469 ... -5.108e-07 -5.108e-07
    mass                (t) float64 2.0 2.0 2.0 2.0 ... 1.659 1.658 1.658 1.658
    mass_dot            (t) float64 0.0 0.0 0.0 ... -0.003466 -0.003466
    h                   (t) float64 1e+03 1e+03 1e+03 ... 1.7e+03 1.702e+03
    engine_on           (t) bool False False False False ... True True True True
    stage_state         (t) int64 0 0 0 0 0 0 0 0 0 0 0 ... 1 1 1 1 1 1 1 1 1 1
    stage_idx           (t) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0

In [58]:
plt.figure()
result.xii.sel(d_2_0=0).plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
plt.figure()
plt.plot(result.xii.sel(d_2_0=0), result.xii.sel(d_2_0=1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
plt.figure()
result.aii.plot.line(x="t")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:
plt.figure()
result.h.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …